In [37]:
import pandas as pd
import re
from sklearn import metrics
from sklearn.metrics import accuracy_score
#The itertools module includes a set of functions for working with iterable (sequence-like) data sets.
import itertools

In [38]:
df = pd.read_csv(r'D:\Fake News Detection\Data\pre_train_data.csv') 

In [39]:
df.drop(['stt','id','timestamp_post'], axis='columns', inplace=True)

In [40]:
inputs = df.drop('label',axis='columns')
inputs.post_message = inputs.post_message.str.replace("[0-9]","")
inputs.post_message = inputs.post_message.str.replace("[`~!@#$%^&*()-=+\|?><;:,.{}']","")
inputs.post_message = inputs.post_message.str.replace(" +"," ")
inputs.post_message = inputs.post_message.str.replace("_+","_")
inputs.head()

,post_message,num_like_post,num_comment_post,num_share_post
0,thăng cấp_bậc hàm đối_với cán_bộ chiến sỹ hy_s...,19477,378,173
1,tư_vấn mùa thi cách nộp hồ_sơ để trúng_tuyển c...,11,5,3
2,cơ_quan cạnh_tranh và thị_trường anh quyết_địn...,48,5,19
3,thêm ca tại quảng_nam liên_quan đến hành_khách...,3,0,0
4,trong giờ học_thể dụ ‌ c do thầy_giáo nguyễn_v...,775,0,54


In [41]:
print(inputs.post_message[0])

thăng cấp_bậc hàm đối_với cán_bộ chiến sỹ hy_sinh ở đà nẵng ngày đại_tướng tô lâm bộ_trưởng bộ công_an đã ký quyết_định số qđ bcax thăng cấp_bậc hàm từ đại_úy lên thiếu_tá đối_với đồng_chí đặng_thanh tuấn cùng ngày thiếu_tướng vũ xuân_viên giám_đốc công_an thành_phố đà nẵng ký quyết_định số qđ cathành phố thăng cấp_bậc hàm từ trung_sĩ lên thượng sỹ đối_với đồng_chí võ văn toàn đây là cán_bộ chiến sỹ đã hy_sinh trong quá_trình thực_hiện nhiệm_vụ đảm_bảo an_ninh trật_tự phòng_chống tội_phạm trên địa_bàn thành_phố đà nẵng trước đó vào lúc h ngày công_an thành_phố đà nẵng nhận được tin báo của nhân_dân có nhóm đối_tượng đua xe và cướp_giật người đi đường tại khu_vực quận sơn_trà trong bối_cảnh toàn_quốc thực_hiện cách_ly xã_hội theo chỉ_thị số của thủ_tướng chính_phủ công_an thành_phố đà nẵng đã chỉ_đạo cho công_an quận sơn_trà triển_khai lực_lượng truy bắt nhóm đối_tượng trong quá_trình truy_đuổi các đối_tượng trên đại_úy đặng_thanh_tuấn và trung_sĩ võ văn toàn_bị tai_nạn và hy_sinh ảnh t

In [42]:
from sklearn.model_selection import train_test_split
data_train, data_test, label_train, label_test = train_test_split(inputs, df.label, test_size=0.2)

In [43]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer, TfidfTransformer
vec = CountVectorizer(ngram_range=(1,2))     #max_features=5000
data_train_count = vec.fit_transform(data_train.post_message)
vec.get_feature_names()[:100]
#print(data_train_count.toarray())

['_acid',
 '_acid dạ_dày',
 '_ban',
 '_ban chỉ_đạo',
 '_bước_',
 '_bước_ like',
 '_bộ',
 '_bộ công_thương',
 '_chuỗi',
 '_chuỗi siêu_thị',
 '_chàng',
 '_chàng ca_sĩ',
 '_coronavirus',
 '_coronavirus có_thể',
 '_cp',
 '_cp lại',
 '_cười',
 '_cần',
 '_cần tiếng_nói',
 '_dạ',
 '_dạ vâng',
 '_dịch',
 '_dịch nha',
 '_dữ_liệu',
 '_dữ_liệu bài',
 '_dữ_liệu cho',
 '_dữ_liệu giá',
 '_dữ_liệu phân_tích',
 '_dữ_liệu từ',
 '_hoá',
 '_hoá biochemical',
 '_hình',
 '_hình nguồn',
 '_hóa_an',
 '_hóa_an ên',
 '_hóa_đơn',
 '_hóa_đơn tiền',
 '_hải',
 '_hải chạy',
 '_hậu_lộc',
 '_hậu_lộc chủ',
 '_khí',
 '_khí dung',
 '_kỳ',
 '_kỳ thi_tuyển_sinh',
 '_kỳ_hóa_đơn',
 '_kỳ_hóa_đơn tháng',
 '_liệu',
 '_liệu khoa_học',
 '_lượng',
 '_lượng hàng',
 '_lượng lớn',
 '_mác',
 '_mác và',
 '_nghỉ',
 '_nghỉ nghỉ',
 '_nêu',
 '_nêu ý_kiến',
 '_nữ',
 '_nữ bệnh_nhân',
 '_pháp',
 '_pháp hiện',
 '_pháp hủy_bỏ',
 '_phượng',
 '_phượng phải',
 '_phượng đau_đớn',
 '_quang',
 '_quang có',
 '_sai',
 '_sai bét_nhè',
 '_st',
 '_st thu

In [44]:
data_train_count.shape

(2717, 149630)

In [45]:
from sklearn.naive_bayes import MultinomialNB
NB = MultinomialNB(alpha = 1.0)
NB.fit(data_train_count,label_train)

MultinomialNB()

In [46]:
data_test_count = vec.transform(data_test.post_message)
label_pred = NB.predict(data_test_count)

In [47]:
print('Training size = %d, accuracy = %.2f%%\n' % \
     (data_train_count.shape[0], accuracy_score(label_test, label_pred)*100))
print('Confusion matrix:\n', metrics.confusion_matrix(label_test, label_pred))
print(metrics.classification_report(label_test, label_pred))

Training size = 2717, accuracy = 82.65%

Confusion matrix:
 [[562   6]
 [112   0]]
              precision    recall  f1-score   support

           0       0.83      0.99      0.90       568
           1       0.00      0.00      0.00       112

    accuracy                           0.83       680
   macro avg       0.42      0.49      0.45       680
weighted avg       0.70      0.83      0.76       680



In [48]:
from sklearn.pipeline import Pipeline
clf = Pipeline([
    ('cv', CountVectorizer(ngram_range=(1,3))),
    ('tfidf', TfidfTransformer()),
    ('nb', MultinomialNB())
])
clf.fit(data_train.post_message, label_train)
clf.score(data_test.post_message, label_test)

0.8352941176470589